In [1]:
import numpy as np
import plotly.graph_objects as go
import os
import gzip
import pickle as pkl

def loadall_results(path, horizon, n_folds):
    result = np.zeros( (n_folds, horizon) )
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds):
            try:
                res = pkl.load(f)
                result[i] = res
            except EOFError:
                break
    return result


In [ ]:

n_folds = 96
horizon = 20000

game = 'LE'
task = 'imbalanced'

colors = [  [250,0,0],[250,0,0], [250,0,0], [250,0,0],
            [0,250,0], [0,250,0], [0,250,0], [0,250,0],
            [0,0,250], [0,0,250], [0,0,250],[0,0,250],
            [255,128,0], [255,128,0], [255,128,0], [255,128,0]  ]

labels = [   'RandCBP_1_5',  'RandCBP_18_5', 'RandCBP_116_5','RandCBP_132_5', 
             'RandCBP_1_10',  'RandCBP_18_10', 'RandCBP_116_10','RandCBP_132_10',
             'RandCBP_1_20',  'RandCBP_18_20', 'RandCBP_116_20','RandCBP_132_20', 
             'RandCBP_1_100',  'RandCBP_18_100', 'RandCBP_116_100','RandCBP_132_100'  ]
             
fig1 = go.Figure(    )


for color, label in zip( colors, labels):
    r,g,b = color

    direct = './results/benchmark_randcbp/{}/'.format(game)
    path = os.path.join(direct, '{}_{}_{}_{}.pkl.gz'.format(task,horizon,n_folds,label) )
    result = loadall_results(path, horizon, n_folds)

    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    if 'RandCBP_132' in label:
        fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dot' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ), name='σ = 1/32'.format( label.split('_')[2] ) ) ) 

    elif 'RandCBP_116' in label:
        fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'longdashdot' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ), name='σ = 1/16'.format( label.split('_')[2] ) ) )  

    elif 'RandCBP_18' in label:
        fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dash' ), mode='lines',
                legendgroup='M = {}'.format( label.split('_')[2] ),  name='σ = 1/8'.format( label.split('_')[2] ) ) ) 

    elif 'RandCBP_1' in label:
        fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ),legendgrouptitle_text='M = {}'.format( label.split('_')[2] ), name='σ = 1'.format( label.split('_')[2] ) ) ) 

    fig1.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.05)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False ) )
    
fig1.update_yaxes(range=[0, 50] )
fig1.update_layout(legend= dict(yanchor="top",y=1,xanchor="left",x=0.43),
                    legend_orientation="h",
                    autosize=False,
                    xaxis_title="Sequence",
                    yaxis_title="Regret",
                    margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                    font=dict(size=13,) )
fig1.show()
fig1.write_image("./benchmarkRandCBP_{}_{}.pdf".format(task, game) )
